# RNN前向传播计算过程

In [1]:
import numpy as np
from rnn_utils import *
from public_tests import *

## 1 - 基本RNN的前向传播

<img src="images/RNN.png" style="width:500;height:300px;">
<caption><center><font><b>Figure 1</b>: 基本RNN </center></caption>

### 输入 $x$ 的维度

#### 包含 $n_x$ 个单元的输入
* 对于单个输入示例的单个时间步长，$x^{(i) \langle t \rangle }$ 是一个一维输入向量
* 以语言为例，词汇量为 5000 的语言可以进行独热编码，生成一个包含 5000 个单元的向量。因此，$x^{(i)\langle t \rangle}$ 的形状为 (5000,)
* 此处使用符号 $n_x$ 表示单个训练示例单个时间步长的单元数

#### 大小为 $T_{x}$ 的时间步长
* 循环神经网络有多个时间步长，你可以用 $t$ 来索引它们。
* 由多个时间步长 $T_x$ 组成的训练样本 $x^{(i)}$。 $T_{x}$ 表示最长序列中的时间步长数。

#### 大小为 $m$ 的批次
* 假设我们有一些小批次，每个批次包含 20 个训练样本
* 为了利用向量化的优势，你需要堆叠 20 列 $x^{(i)}$ 个样本
* 例如，这个张量的形状为 (5000,20,10)
* 你将使用 $m$ 表示训练样本的数量
* 因此，小批次的形状为 $(n_x,m,T_x)$

#### 形状为 $(n_{x},m,T_{x})$ 的三维张量
* 形状为 $(n_x,m,T_x)$ 的三维张量 $x$ 表示输入到 RNN 的输入 $x$。

#### 为每个时间步取一个二维切片：$x^{\langle t \rangle}$
* 在每个时间步，你将使用一个小批量的训练样本（而不仅仅是单个样本）。
* 因此，对于每个时间步 $t$，你将使用一个形状为 $(n_x,m)$ 的二维切片。
* 这个二维切片被称为 $x^{\langle t \rangle}$。代码中的变量名为 `xt`。

### 隐藏状态 $a$ 的定义

* 从一个时间步传递到 RNN 的激活值 $a^{\langle t \rangle}$ 被称为“隐藏状态”。

### 隐藏状态 $a$ 的维度

* 与输入张量 $x$ 类似，单个训练样本的隐藏状态是一个长度为 $n_{a}$ 的向量。
* 如果包含一个包含 $m$ 个训练样本的小批量，则小批量的形状为 $(n_{a},m)$。
* 如果包含时间步长维度，则隐藏状态的形状为 $(n_{a}, m, T_x)$。
* 你将循环遍历索引为 $t$ 的时间步长，并使用三维张量的二维切片。
* 此二维切片称为 $a^{\langle t \rangle}$。
* 在代码中，使用的变量名称是 `a_prev` 或 `a_next`，具体取决于所实现的函数。
* 此二维切片的形状为 $(n_{a}, m)$。

### 预测维度 $\hat{y}$
* 与输入和隐藏状态类似，$\hat{y}$ 是一个三维张量，形状为 $(n_{y}, m, T_{y})$
* $n_{y}$：表示预测的向量中的单元数
* $m$：小批量中的样本数
* $T_{y}$：预测的时间步数
* 对于单个时间步 $t$，二维切片 $\hat{y}^{\langle t \rangle}$ 的形状为 $(n_{y}, m)$
* 代码中的变量名称如下：
- `y_pred`: $\hat{y}$
- `yt_pred`: $\hat{y}^{\langle t \rangle}$

以下是实现 RNN 的方法：

### 步骤：
1. 实现 RNN 一个时间步所需的计算。
2. 实现一个循环，遍历 $T_x$ 个时间步，以便一次处理所有输入。

<a name='1-1'></a>
### 1.1 - RNN 单元

你可以将循环神经网络视为单个单元的重复使用。首先，你将实现单个时间步长的计算。下图描述了 RNN 单元单个时间步长的操作：

<img src="images/rnn_step_forward_figure2_v3a.png" style="width:700px;height:300px;">
<caption><center><b>图 2</b>：基本 RNN 单元。输入 $x^{\langle t \rangle}$（当前输入）和 $a^{\langle t - 1\rangle}$（包含过去信息的先前隐藏状态），输出 $a^{\langle t \rangle}$，该输出将传递给下一个 RNN 单元，并用于预测 $\hat{y}^{\langle t \rangle}$
</center></caption>

**`RNN 单元` 与 `RNN_cell_forward`**：
* 注意，RNN 单元输出隐藏状态 $a^{\langle t \rangle}$。
* `RNN cell` 在图中显示为实线内框
* 你将要实现的函数 `rnn_cell_forward` 还会计算预测 $\hat{y}^{\langle t \rangle}$
* `RNN_cell_forward` 在图中显示为虚线外框

### 练习 1 - rnn_cell_forward

实现图 2 中描述的 RNN 单元。

**说明**:
1. 使用 tanh 激活函数计算隐藏状态：$a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a)$
2. 使用新的隐藏状态 $a^{\langle t \rangle}$ 计算预测值 $\hat{y}^{\langle t \rangle} = softmax(W_{ya} a^{\langle t \rangle} + b_y)$。 （已提供函数 `softmax`）
3. 将 $(a^{\langle t \rangle}, a^{\langle t-1 \rangle}, x^{\langle t \rangle}, 参数)$ 存储在 `cache` 中
4. 返回 $a^{\langle t \rangle}$ 、 $\hat{y}^{\langle t \rangle}$ 和 `cache`

#### 额外提示
* 激活函数：np.tanh
* 有现有的 `softmax` 函数提供使用。它位于文件 'rnn_utils.py' 中，并且已导入。
* 矩阵乘法：np.dot

In [2]:
def rnn_cell_forward(xt, a_prev, parameters):
    """
    实现 RNN 单元的单步前向传播，如图 (2) 所示。

    参数：
    xt - 时间步长 "t" 的输入数据，numpy 数组，形状为 (n_x, m)。
    a_prev - 时间步长 "t-1" 的隐藏状态，numpy 数组，形状为 (n_a, m)。
    parameters - 包含以下内容的 Python 字典：
        Wax - 与输入相乘的权重矩阵，numpy 数组，形状为 (n_a, n_x)。
        Waa - 与隐藏状态相乘的权重矩阵，numpy 数组，形状为 (n_a, n_a)。
        Wya - 关联隐藏状态和输出的权重矩阵，numpy 数组，形状为 (n_y, n_a)。
        ba - 偏差，numpy 数组，形状为 (n_a, 1)。
        by - 关联隐藏状态和输出的偏差，numpy 数组，形状为(n_y, 1)

    返回：
    a_next - 下一个隐藏状态，形状为 (n_a, m)
    yt_pred - 时间步 "t" 的预测，形状为 (n_y, m) 的 NumPy 数组
    cache - 反向传播所需值的元组，包含 (a_next, a_prev, xt, parameters)
    """

    # todo: 1. 从“parameters”中提取权重+偏置
    Wax, Waa, Wya, ba, by = parameters["Wax"], parameters["Waa"], parameters["Wya"], parameters["ba"], parameters["by"]

    # todo: 2. 使用上述公式计算下一个激活状态
    a_next = np.tanh(Waa @ a_prev + Wax @ xt + ba)

    # todo: 3. 使用公式计算当前单元的输出
    yt_pred = softmax(Wya @ a_next + by)

    # 将反向传播所需的值存储在缓存中
    cache = (a_next, a_prev, xt, parameters)

    return a_next, yt_pred, cache

In [3]:
np.random.seed(1)
xt_tmp = np.random.randn(3, 10)
a_prev_tmp = np.random.randn(5, 10)
parameters_tmp = {}
parameters_tmp['Waa'] = np.random.randn(5, 5)
parameters_tmp['Wax'] = np.random.randn(5, 3)
parameters_tmp['Wya'] = np.random.randn(2, 5)
parameters_tmp['ba'] = np.random.randn(5, 1)
parameters_tmp['by'] = np.random.randn(2, 1)

a_next_tmp, yt_pred_tmp, cache_tmp = rnn_cell_forward(xt_tmp, a_prev_tmp, parameters_tmp)
print("a_next[4] = \n", a_next_tmp[4])
print("a_next.shape = \n", a_next_tmp.shape)
print("yt_pred[1] =\n", yt_pred_tmp[1])
print("yt_pred.shape = \n", yt_pred_tmp.shape)

# UNIT TESTS
rnn_cell_forward_tests(rnn_cell_forward)

a_next[4] = 
 [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape = 
 (5, 10)
yt_pred[1] =
 [0.9888161  0.01682021 0.21140899 0.36817467 0.98988387 0.88945212
 0.36920224 0.9966312  0.9982559  0.17746526]
yt_pred.shape = 
 (2, 10)
All tests passed


**Expected Output**: 
```text
a_next[4] = 
 [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape = 
 (5, 10)
yt_pred[1] =
 [ 0.9888161   0.01682021  0.21140899  0.36817467  0.98988387  0.88945212
  0.36920224  0.9966312   0.9982559   0.17746526]
yt_pred.shape = 
 (2, 10)

```

### 1.2 - RNN 前向传递

- 循环神经网络 (RNN) 是刚刚构建的 RNN 单元的重复。
- 如果你的输入数据序列长度为 10 个时间步，那么你将重复使用 RNN 单元 10 次。
- 每个单元在每个时间步接受两个输入：
- $a^{\langle t-1 \rangle}$：来自前一个单元的隐藏状态
- $x^{\langle t \rangle}$：当前时间步的输入数据
- 它在每个时间步有两个输出：
- 隐藏状态 ($a^{\langle t \rangle}$)
- 预测 ($y^{\langle t \rangle}$)
- 权重和偏差 $(W_{aa}, W_{ax}, b_{a}, W_{ay}, b_{y})$ 在每个时间步重复使用。
- 它们在调用 `rnn_cell_forward` 函数时保存在 `parameters` 字典中。

<img src="images/rnn_forward_sequence_figure3_v3a.png" style="zoom: 150%;">

<caption><center><b>图 3</b>：基本 RNN。输入序列 $x = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ 沿 $T_x$ 个时间步进行。网络输出 $y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$。</center></caption>

### Exercise 2 - rnn_forward（RNN 前向传播）

**指示说明：**

- 创建一个三维的全零数组 $a$，形状为 $(n_a, m, T_x)$，用于存储 RNN 在所有时间步计算得到的隐藏状态（hidden states）。
- 创建一个三维的全零数组 $\hat{y}$，形状为 $(n_y, m, T_x)$，用于存储每个时间步的预测值。
  - 注意：此处 $T_y = T_x$（即预测序列与输入序列具有相同的时间步数）。
- 初始化二维隐藏状态 `a_next`，将其设为初始隐藏状态 $a_0$。
- 对于每一个时间步 $t$：

  - 取出单个时间步的输入片段 $x^{\langle t \rangle}$，即从 $x$ 中截取出对应时间步 $t$ 的二维切片：
    - $x^{\langle t \rangle}$ 的形状为 $(n_x, m)$
    - 整个输入 $x$ 的形状为 $(n_x, m, T_x)$

  - 调用 `rnn_cell_forward`，更新当前时间步的二维隐藏状态 $a^{\langle t \rangle}$（变量名 `a_next`）、预测值 $\hat{y}^{\langle t \rangle}$（变量名 `yt_pred`），并获得对应的缓存 `cache`。
    - $a^{\langle t \rangle}$ 的形状为 $(n_a, m)$

  - 将当前的二维隐藏状态存入三维张量 $a$ 的第 $t$ 个位置。
    - $a$ 的形状为 $(n_a, m, T_x)$

  - 将当前的二维预测值 $\hat{y}^{\langle t \rangle}$（变量名 `yt_pred`）存入三维张量 $\hat{y}_{pred}$ 的第 $t$ 个位置。
    - $\hat{y}^{\langle t \rangle}$ 的形状为 $(n_y, m)$
    - $\hat{y}$ 的形状为 $(n_y, m, T_x)$

  - 将本时间步的缓存 `cache` 添加进缓存列表 `caches`。

- 最后返回三维张量 $a$、$\hat{y}$ 以及缓存列表 `caches`。


In [4]:
def rnn_forward(x, a0, parameters):
    """
    实现循环神经网络的前向传播。

    参数：
    x - 每个时间步的输入数据，形状为 (n_x, m, T_x)。
    a0 - 初始隐藏状态，形状为 (n_a, m)。
    parameters - 包含以下内容的 Python 字典：
        Waa - 权重矩阵，用于乘以隐藏状态，numpy 数组，形状为 (n_a, n_a)。
        Wax - 权重矩阵，用于乘以输入，numpy 数组，形状为 (n_a, n_x)。
        Wya - 权重矩阵，用于关联隐藏状态和输出，numpy 数组，形状为 (n_y, n_a)。
        ba - 偏差，numpy 数组，形状为 (n_a, 1)。
        by - 偏差，用于关联隐藏状态和输出，numpy 数组，形状为 (n_y, 1)。

    返回：
    a - 每个时间步的隐藏状态时间步长，numpy 数组，形状为 (n_a, m, T_x)
    y_pred - 每个时间步长的预测值，numpy 数组，形状为 (n_y, m, T_x)
    caches - 反向传播所需值的元组，包含 (caches 列表, x)
    """

    # 初始化“caches”，其中包含所有 caches 的列表
    caches = []

    # 从 x 和 parameters["Wya"] 的形状中获取维度
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape

    ### 从此处开始代码###

    # 用零初始化“a”和“y_pred”
    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))

    #初始化 a_next
    a_next = a0         # na x m

    # TODO: 循环遍历所有时间步
    for t in range(T_x):
        # 1. 更新下一个隐藏状态，计算预测，获取缓存（约 1 行）
        a_next, yt_pred, cache = rnn_cell_forward(x[:, : ,t], a_next, parameters)
        # 2. 将新的“下一个”隐藏状态的值保存在 a 中（约 1 行）
        a[:, :, t] = a_next
        # 3. 将预测值保存在 y 中（约 1 行）
        y_pred[:, :, t] = yt_pred
        # 4. 将“cache”附加到“caches”中（约 1 行）
        caches.append(cache)
    # 将反向传播所需的值存储在缓存中
    caches = (caches, x)
    return a, y_pred, caches

In [5]:
np.random.seed(1)
x_tmp = np.random.randn(3, 10, 4)
a0_tmp = np.random.randn(5, 10)
parameters_tmp = {}
parameters_tmp['Waa'] = np.random.randn(5, 5)
parameters_tmp['Wax'] = np.random.randn(5, 3)
parameters_tmp['Wya'] = np.random.randn(2, 5)
parameters_tmp['ba'] = np.random.randn(5, 1)
parameters_tmp['by'] = np.random.randn(2, 1)

a_tmp, y_pred_tmp, caches_tmp = rnn_forward(x_tmp, a0_tmp, parameters_tmp)
print("a[4][1] = \n", a_tmp[4][1])
print("a.shape = \n", a_tmp.shape)
print("y_pred[1][3] =\n", y_pred_tmp[1][3])
print("y_pred.shape = \n", y_pred_tmp.shape)
print("caches[1][1][3] =\n", caches_tmp[1][1][3])
print("len(caches) = \n", len(caches_tmp))

#UNIT TEST    
rnn_forward_test(rnn_forward)

a[4][1] = 
 [-0.99999375  0.77911235 -0.99861469 -0.99833267]
a.shape = 
 (5, 10, 4)
y_pred[1][3] =
 [0.79560373 0.86224861 0.11118257 0.81515947]
y_pred.shape = 
 (2, 10, 4)
caches[1][1][3] =
 [-1.1425182  -0.34934272 -0.20889423  0.58662319]
len(caches) = 
 2
All tests passed


**Expected Output**:

```text
a[4][1] = 
 [-0.99999375  0.77911235 -0.99861469 -0.99833267]
a.shape = 
 (5, 10, 4)
y_pred[1][3] =
 [ 0.79560373  0.86224861  0.11118257  0.81515947]
y_pred.shape = 
 (2, 10, 4)
caches[1][1][3] =
 [-1.1425182  -0.34934272 -0.20889423  0.58662319]
len(caches) = 
 2
```

## 2 - Long Short-Term Memory (LSTM) 网络

下图展示了 LSTM 单元的操作：

<img src="images/LSTM_figure4_v3a.png">
<caption><center><b>图 4</b>：LSTM 单元。它会在每个时间步长跟踪并更新“单元状态”，即记忆变量 $c^{\langle t \rangle}$，该变量可能与 $a^{\langle t \rangle}$ 不同。
注意，$softmax^{}$ 包含一个全连接层和一个 softmax 层。</center></caption>

与上面的 RNN 示例类似，你将首先实现单个时间步长的 LSTM 单元。然后，你将在“for 循环”中迭代调用它，使其以 $T_x$ 个时间步长处理输入。

### 门和状态概述

#### 遗忘门 $\mathbf{\Gamma}_{f}$

* 假设你正在阅读一段文本中的单词，并计划使用 LSTM 来跟踪语法结构，例如主语是单数（“puppy”）还是复数（“puppies”）。
* 如果主语的状态发生变化（从单数词变为复数词），先前状态的记忆就会变得过时，因此你会“忘记”该过时状态。
* “遗忘门”是一个包含 0 到 1 之间值的张量。
* 如果遗忘门中某个单元的值接近于 0，LSTM 就会“忘记”先前单元状态对应单元中存储的状态。
* 如果遗忘门中某个单元的值接近于 1，LSTM 会基本记住存储状态中对应的值。

##### 方程

$$\mathbf{\Gamma}_f^{\langle t \rangle} = \sigma(\mathbf{W}_f[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_f)\tag{1} $$

##### 等式的解释：

* $\mathbf{W_{f}}$ 包含控制遗忘门行为的权重。
* 前一个时间步的隐藏状态 $[a^{\langle t-1 \rangle}$ 和当前时间步的输入 $x^{\langle t \rangle}]$ 连接在一起并乘以 $\mathbf{W_{f}}$。
* 使用 S 型函数使每个门张量 $\mathbf{\Gamma}_f^{\langle t \rangle}$ 的值介于 0 到 1 之间。
* 遗忘门 $\mathbf{\Gamma}_f^{\langle t \rangle}$ 的维度与前一个cell状态 $c^{\langle t-1 \rangle}$ 相同。
* 这意味着两者可以逐个元素相乘。
* 将张量 $\mathbf{\Gamma}_f^{\langle t \rangle} * \mathbf{c}^{\langle t-1 \rangle}$ 相乘就像在前一个cell状态上应用了一个掩码。
* 如果 $\mathbf{\Gamma}_f^{\langle t \rangle}$ 中的单个值为 0 或接近于 0，则乘积接近于 0。
* 这使得 $\mathbf{c}^{\langle t-1 \rangle}$ 中相应单元存储的信息不会被记住用于下一个时间步。
* 类似地，如果一个值接近于 1，则乘积接近于前一个单元状态的原始值。
* LSTM 会保留 $\mathbf{c}^{\langle t-1 \rangle}$ 相应单元的信息，以便在下一个时间步使用。

##### 代码中的变量名称
代码中的变量名称与方程式类似，但略有不同。
* `Wf`: 遗忘门权重 $\mathbf{W}_{f}$
* `bf`: 遗忘门偏置 $\mathbf{b}_{f}$
* `ft`: 遗忘门 $\Gamma_f^{\langle t \rangle}$

#### 候选值 $\tilde{\mathbf{c}}^{\langle t \rangle}$
* 候选值是一个张量，包含当前时间步的信息，这些信息**可能**存储在当前单元状态 $\mathbf{c}^{\langle t \rangle}$ 中。
* 候选值中哪些部分会被传递取决于更新（输入）门。
* 候选值是一个张量，其值范围从 -1 到 1。
* 波浪号“~”用于区分候选值 $\tilde{\mathbf{c}}^{\langle t \rangle}$ 和单元状态 $\mathbf{c}^{\langle t \rangle}$。

##### 方程
$$\mathbf{\tilde{c}}^{\langle t \rangle} = \tanh\left( \mathbf{W}_{c} [\mathbf{a}^{\langle t - 1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{c} \right) \tag{3}$$

##### 方程式解释
* *tanh* 函数的值为 -1 到 1 之间的值。

##### 代码中的变量名
* `cct`：候选值 $\mathbf{\tilde{c}}^{\langle t \rangle}$

#### 输入门 $\mathbf{\Gamma}_{i}$

* 使用输入门来决定将候选张量 $\tilde{\mathbf{c}}^{\langle t \rangle}$ 的哪些部分添加到单元状态 $c^{\langle t \rangle}$。
* 输入门决定将“候选”张量 $\tilde{\mathbf{c}}^{\langle t \rangle}$ 的哪些部分传递到单元状态 $\mathbf{c}^{\langle t \rangle}$。
* 输入门是一个包含 0 到 1 之间值的张量。
* 当输入门中的某个单元接近于 1 时，它允许候选值 $\tilde{\mathbf{c}}^{\langle t \rangle}$ 的值传递到隐藏状态 $\mathbf{c}^{\langle t \rangle}$。
* 当输入门中的某个单元接近于 0 时，它会阻止候选值中相应的值传递到隐藏状态。
* 注意，为了遵循文献中的惯例，我们使用下标“i”而不是“u”。

##### 公式

$$\mathbf{\Gamma}_i^{\langle t \rangle} = \sigma(\mathbf{W}_i[a^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_i)\tag{2} $$

##### 公式解释

* 与遗忘门类似，这里 $\mathbf{\Gamma}_i^{\langle t \rangle}$，S 型函数产生的值介于 0 到 1 之间。
* 输入门与候选元素逐个相乘，该乘积（$\mathbf{\Gamma}_{i}^{\langle t \rangle} * \tilde{c}^{\langle t \rangle}$）用于确定cell状态 $\mathbf{c}^{\langle t \rangle}$。

##### 代码中的变量名称（请注意，它们与方程式不同）
在代码中，你将使用学术文献中找到的变量名称。这些变量不使用“u”表示“更新”。
* `Wi` 是输入门权重 $\mathbf{W}_i$（不是“Wu”）
* `bi` 是输入门偏置 $\mathbf{b}_i$（不是“bu”）
* `it` 是输入门 $\mathbf{\Gamma}_i^{\langle t \rangle}$（不是“ut”）

#### 单元状态 $\mathbf{c}^{\langle t \rangle}$

* 单元状态是传递到未来时间步的“记忆”。
* 新的单元状态 $\mathbf{c}^{\langle t \rangle}$ 是先前单元状态和候选值的组合。

##### 方程

$$ \mathbf{c}^{\langle t \rangle} = \mathbf{\Gamma}_f^{\langle t \rangle}* \mathbf{c}^{\langle t-1 \rangle} + \mathbf{\Gamma}_{i}^{\langle t \rangle} *\mathbf{\tilde{c}}^{\langle t \rangle} \tag{4} $$

##### 等式的解释
* 之前的cell状态 $\mathbf{c}^{\langle t-1 \rangle}$ 由遗忘门调整（加权） $\mathbf{\Gamma}_{f}^{\langle t \rangle}$
* 和候选值 $\tilde{\mathbf{c}}^{\langle t \rangle}$，由更新门 $\mathbf{\Gamma}_{i}^{\langle t \rangle}$ 调整（加权）。

##### 代码中的变量名称和形状
* `c`：cell状态，包含所有时间步，$\mathbf{c}$ 形状 $(n_{a}, m, T_x)$
* `c_next`：新的（下一个）cell状态，$\mathbf{c}^{\langle t \rangle}$ 形状 $(n_{a}, m)$
* `c_prev`：前一个cell状态，$\mathbf{c}^{\langle t-1 \rangle}$，形状 $(n_{a}, m)$

#### 输出门 $\mathbf{\Gamma}_{o}$

* 输出门决定将什么作为时间步长的预测（输出）发送。
* 输出门与其他门类似，其值的范围为 0 到 1。

##### 公式

$$ \mathbf{\Gamma}_o^{\langle t \rangle}= \sigma(\mathbf{W}_o[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{o})\tag{5}$$

##### 公式解释
* 输出门由前一个隐藏状态 $\mathbf{a}^{\langle t-1 \rangle}$ 和当前输入 $\mathbf{x}^{\langle t \rangle}$ 决定。
* S 型函数使门的值范围为 0 到 1。

##### 变量代码中的名称
* `Wo`：输出门权重，$\mathbf{W_o}$
* `bo`：输出门偏置，$\mathbf{b_o}$
* `ot`：输出门，$\mathbf{\Gamma}_{o}^{\langle t \rangle}$

#### 隐藏状态 $\mathbf{a}^{\langle t \rangle}$

* 隐藏状态会传递到 LSTM 单元的下一个时间步。
* 它用于确定下一个时间步的三个门控（$\mathbf{\Gamma}_{f}, \mathbf{\Gamma}_{u}, \mathbf{\Gamma}_{o}$）。
* 隐藏状态也用于预测 $y^{\langle t \rangle}$。

##### 公式

$$ \mathbf{a}^{\langle t \rangle} = \mathbf{\Gamma}_o^{\langle t \rangle} * \tanh(\mathbf{c}^{\langle t \rangle})\tag{6} $$

##### 公式解释
* 隐藏状态 $\mathbf{a}^{\langle t \rangle}$ 由单元状态 $\mathbf{c}^{\langle t \rangle}$ 和输出门 $\mathbf{\Gamma}_{o}$ 共同决定。
* 单元状态通过 `tanh` 函数传递，将值缩放到 -1 到 1 之间。
* 输出门的作用类似于“掩码”，它要么保留 $\tanh(\mathbf{c}^{\langle t \rangle})$ 的值，要么阻止这些值包含在隐藏状态 $\mathbf{a}^{\langle t \rangle}$ 中。

##### 代码中的变量名称和形状
* `a`：隐藏状态，包括时间步长。$\mathbf{a}$ 的形状为 $(n_{a}, m, T_{x})$。
* `a_prev`：上一时间步长的隐藏状态。$\mathbf{a}^{\langle t-1 \rangle}$ 的形状为 $(n_{a}, m)$。
* `a_next`：下一时间步长的隐藏状态。 $\mathbf{a}^{\langle t \rangle}$ 的形状为 $(n_{a}, m)$

#### 预测 $\mathbf{y}^{\langle t \rangle}_{pred}$
* 本用例中的预测是分类，因此你将使用 softmax。

公式如下：
$$\mathbf{y}^{\langle t \rangle}_{pred} = \textrm{softmax}(\mathbf{W}_{y} \mathbf{a}^{\langle t \rangle} + \mathbf{b}_{y})$$

##### 代码中的变量名称和形状
* `y_pred`：预测，包含所有时间步长。$\mathbf{y}_{pred}$ 的形状为 $(n_{y}, m, T_{x})$。注意，本例中为 $(T_{y} = T_{x})$。
* `yt_pred`：当前时间步长 $t$ 的预测。$\mathbf{y}^{\langle t \rangle}_{pred}$ 的形状为 $(n_{y}, m)$

### 2.1 - LSTM 单元

### 练习 3 - lstm_cell_forward

实现图中所示的 LSTM 单元。

**说明**：
1. 将隐藏状态 $a^{\langle t-1 \rangle}$ 和输入 $x^{\langle t \rangle}$ 连接成一个矩阵：

$$concat = \begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle} \end{bmatrix}$$

2. 计算门控、隐藏状态和单元状态的所有公式（1 到 6）。
3. 计算预测值 $y^{\langle t \rangle}$。


#### 其他提示
* 函数 `sigmoid()` 和 `softmax` 是从 `rnn_utils.py` 导入的。
* 请注意，变量名 `Wi` 和 `bi` 分别表示**更新**门的权重和偏差。此函数中没有名为 "Wu" 或 "bu" 的变量。

In [6]:
def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    """
    实现如图 所示的 LSTM 单元的单步前向传播。

    参数：
    xt - 时间步 "t" 的输入数据，numpy 数组，形状为 (n_x, m)。
    a_prev - 时间步 "t-1" 的隐藏状态，numpy 数组，形状为 (n_a, m)。
    c_prev - 时间步 "t-1" 的记忆状态，numpy 数组，形状为 (n_a, m)。
    parameters - 包含以下内容的 Python 字典：
        Wf - 遗忘门的权重矩阵，numpy 数组，形状为 (n_a, n_a + n_x)。
        bf - 遗忘门的偏差，numpy 数组，形状为 (n_a, 1)。
        Wi - 更新门的权重矩阵，numpy 数组，形状为 (n_a, n_a + n_x)。
        bi - 更新门的偏差。 numpy 数组，形状为 (n_a, 1)
        Wc - 第一个 tanh 函数的权重矩阵，numpy 数组，形状为 (n_a, n_a + n_x)
        bc - 第一个 tanh 函数的偏差，numpy 数组，形状为 (n_a, 1)
        Wo - 输出门的权重矩阵，numpy 数组，形状为 (n_a, n_a + n_x)
        bo - 输出门的偏差，numpy 数组，形状为 (n_a, 1)
        Wy - 隐藏状态与输出之间的权重矩阵，numpy 数组，形状为 (n_y, n_a)
        by - 隐藏状态与输出之间的偏差，numpy 数组，形状为 (n_y, 1)

    返回：
    a_next - 下一个隐藏状态，形状为 (n_a, m)
    c_next - 下一个记忆状态，形状为 (n_a, m)
    yt_pred - 时间步的预测"t", numpy 数组，形状为 (n_y, m)
    cache - 反向传播所需值的元组，包含 (a_next, c_next, a_prev, c_prev, xt, parameters)

    注意：ft/it/ot 分别代表遗忘/更新/输出门，cct 代表候选值（波浪号 c），
    c 代表单元状态（记忆）
    """

    # 从 "parameters" 中检索参数
    Wf = parameters["Wf"] # 遗忘门权重
    bf = parameters["bf"]
    Wi = parameters["Wi"] # 更新门权重（注意变量名）
    bi = parameters["bi"] # （注意变量名）
    Wc = parameters["Wc"] # 候选值权重
    bc = parameters["bc"]
    Wo = parameters["Wo"] # 输出门权重
    bo = parameters["bo"]
    Wy = parameters["Wy"] # 预测权重
    by = parameters["by"]

    # 从 xt 和 Wy 的形状中获取维度
    n_x, m = xt.shape
    n_y, n_a = Wy.shape

    ### 从此处开始代码###
    # todo：1. 连接 a_prev 和 xt
    concat_a_xt = np.concatenate((a_prev, xt), axis=0)      # (na+nx) * m


    # todo：2. 使用公式计算 ft、it、cct、c_next、ot、a_next 的值
    ft = sigmoid(Wf @ concat_a_xt + bf)
    it = sigmoid(Wi @ concat_a_xt + bi)
    cct = np.tanh(Wc @ concat_a_xt + bc)
    c_next = ft * c_prev + it * cct
    ot = sigmoid(Wo @ concat_a_xt + bo)
    a_next = ot * np.tanh(c_next)

    # todo: 3. 计算 LSTM 单元的预测值（约 1 行）
    yt_pred = softmax(Wy @ a_next + by)
    ### 代码至此结束 ###

    # 将反向传播所需的值存储在缓存中
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)

    return a_next, c_next, yt_pred, cache

In [7]:
np.random.seed(1)
xt_tmp = np.random.randn(3, 10)
a_prev_tmp = np.random.randn(5, 10)
c_prev_tmp = np.random.randn(5, 10)
parameters_tmp = {}
parameters_tmp['Wf'] = np.random.randn(5, 5 + 3)
parameters_tmp['bf'] = np.random.randn(5, 1)
parameters_tmp['Wi'] = np.random.randn(5, 5 + 3)
parameters_tmp['bi'] = np.random.randn(5, 1)
parameters_tmp['Wo'] = np.random.randn(5, 5 + 3)
parameters_tmp['bo'] = np.random.randn(5, 1)
parameters_tmp['Wc'] = np.random.randn(5, 5 + 3)
parameters_tmp['bc'] = np.random.randn(5, 1)
parameters_tmp['Wy'] = np.random.randn(2, 5)
parameters_tmp['by'] = np.random.randn(2, 1)

a_next_tmp, c_next_tmp, yt_tmp, cache_tmp = lstm_cell_forward(xt_tmp, a_prev_tmp, c_prev_tmp, parameters_tmp)

print("a_next[4] = \n", a_next_tmp[4])
print("a_next.shape = ", a_next_tmp.shape)
print("c_next[2] = \n", c_next_tmp[2])
print("c_next.shape = ", c_next_tmp.shape)
print("yt[1] =", yt_tmp[1])
print("yt.shape = ", yt_tmp.shape)
print("cache[1][3] =\n", cache_tmp[1][3])
print("len(cache) = ", len(cache_tmp))

# UNIT TEST
lstm_cell_forward_test(lstm_cell_forward)

a_next[4] = 
 [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
a_next.shape =  (5, 10)
c_next[2] = 
 [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
c_next.shape =  (5, 10)
yt[1] = [0.79913913 0.15986619 0.22412122 0.15606108 0.97057211 0.31146381
 0.00943007 0.12666353 0.39380172 0.07828381]
yt.shape =  (2, 10)
cache[1][3] =
 [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
len(cache) =  10
All tests passed


**Expected Output**:

```text
a_next[4] = 
 [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
a_next.shape =  (5, 10)
c_next[2] = 
 [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
c_next.shape =  (5, 10)
yt[1] = [ 0.79913913  0.15986619  0.22412122  0.15606108  0.97057211  0.31146381
  0.00943007  0.12666353  0.39380172  0.07828381]
yt.shape =  (2, 10)
cache[1][3] =
 [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
len(cache) =  10
```

### 2.2 - LSTM 的前向传递

现在您已经实现了 LSTM 的一个步骤，您可以使用 for 循环对其进行迭代，以处理 $T_x$ 个输入序列。

<img src="images/LSTM_rnn.png" style="width:500;height:300px;">
<caption><center><b>图 5</b>：多个时间步长的 LSTM。</center></caption>

<a name='ex-4'></a>
### 练习 4 - lstm_forward

实现 `lstm_forward()` 函数，以在 $T_x$ 个时间步长上运行 LSTM。

**说明**
* 根据变量 `x` 和 `parameters` 的形状获取维度 $n_x, n_a, n_y, m, T_x$
* 初始化三维张量 $a$、$c$ 和 $y$
- $a$：隐藏状态，形状为 $(n_{a}, m, T_{x})$
- $c$：单元状态，形状为 $(n_{a}, m, T_{x})$
- $y$：预测，形状为 $(n_{y}, m, T_{x})$（注意，本例中 $T_{y} = T_{x}$）
- **注意** 将一个变量设置为另一个变量相等是一种“引用复制”。换句话说，不要使用 `c = a'，否则这两个变量将指向同一个底层变量。
* 初始化二维张量 $a^{\langle t \rangle}$
- $a^{\langle t \rangle}$ 存储时间步 $t$ 的隐藏状态。变量名为 `a_next`。
- $a^{\langle 0 \rangle}$ 是时间步 0 的初始隐藏状态，在调用函数时传入。变量名为 `a0`。
- $a^{\langle t \rangle}$ 和 $a^{\langle 0 \rangle}$ 表示单个时间步，因此它们的形状均为 $(n_{a}, m)$。
- 通过将 $a^{\langle t \rangle}$ 设置为传入函数的初始隐藏状态 ($a^{\langle 0 \rangle}$) 来初始化 $a^{\langle t \rangle}$。
* 用零初始化 $c^{\langle t \rangle}$。
- 变量名为 `c_next`
- $c^{\langle t \rangle}$ 表示单个时间步，因此其形状为 $(n_{a}, m)$
- **注意**：将 `c_next` 创建为独立的变量，并在内存中拥有自己的位置。不要将其初始化为三维张量 $c$ 的切片。换句话说，**不要**执行 `c_next = c[:,:,0]`。
* 对于每个时间步，执行以下操作：
- 从三维张量 $x$ 中，获取时间步 $t$ 的二维切片 $x^{\langle t \rangle}$
- 调用之前定义的 `lstm_cell_forward` 函数，获取隐藏状态、单元状态、预测和缓存
- 将隐藏状态、单元状态和预测（二维张量）存储在三维张量中
- 将缓存附加到缓存列表中

In [8]:
def lstm_forward(x, a0, parameters):
    """
    使用图所示的 LSTM 单元实现循环神经网络的前向传播。

    参数：
    x - 每个时间步的输入数据，形状为 (n_x, m, T_x)。
    a0 - 初始隐藏状态，形状为 (n_a, m)。
    parameters - 包含以下内容的 Python 字典：
        Wf - 遗忘门的权重矩阵，形状为 (n_a, n_a + n_x) 的 NumPy 数组。
        bf - 遗忘门的偏差，形状为 (n_a, 1) 的 NumPy 数组。
        Wi - 更新门的权重矩阵，形状为 (n_a, n_a + n_x) 的 NumPy 数组。
        bi - 更新门的偏差，形状为 (n_a, 1) 的 NumPy 数组。
        Wc - 第一个“tanh”函数的权重矩阵，形状为 (n_a, n_a + n_x) 的 NumPy 数组。
        bc -- 第一个 tanh 函数的偏差，numpy 数组，形状为 (n_a, 1)
        Wo -- 输出门的权重矩阵，numpy 数组，形状为 (n_a, n_a + n_x)
        bo -- 输出门的偏差，numpy 数组，形状为 (n_a, 1)
        Wy -- 隐藏状态与输出之间的权重矩阵，numpy 数组，形状为 (n_y, n_a)
        by -- 隐藏状态与输出之间的偏差，numpy 数组，形状为 (n_y, 1)

    返回：
    a -- 每个时间步的隐藏状态，numpy 数组，形状为 (n_a, m, T_x)
    y -- 每个时间步的预测值，numpy 数组，形状为 (n_y, m, T_x)
    c -- 单元状态的值，numpy 数组，形状为 (n_a, m, T_x)
    caches -- 反向传播所需值的元组包含（所有缓存列表，x）
    """

    # 初始化“caches”，它将跟踪所有缓存的列表
    caches = []

    ### 从此处开始代码 ###
    Wy = parameters['Wy'] # 将 parameters['Wy'] 保存到局部变量Wy中
    # 从 x 和 parameters['Wy'] 的形状中获取维度
    n_x, m, T_x = x.shape
    n_y, n_a = Wy.shape

    # 用零初始化 "a", "c" 和 "y"
    a = np.zeros((n_a, m, T_x))
    c = np.zeros((n_a, m, T_x))
    y = np.zeros((n_y, m, T_x))

    # 初始化 a_next 和c_next
    a_next = a0
    c_next = np.zeros((n_a, m))

    # 循环遍历所有时间步
    for t in range(T_x):
        # todo: 1. 在时间步 t 从三维输入 x 获取二维切片 xt
        xt = x[:, :, t]
        # todo: 2.更新下一个隐藏状态、下一个记忆状态，计算预测，获取缓存
        a_next, c_next, yt_pred, cache = lstm_cell_forward(xt, a_next, c_next, parameters)
        # todo: 3. 将新的“下一个”隐藏状态的值保存在 a 中
        a[:, :, t] = a_next
        # todo: 4. 保存下一个单元状态的值
        c[:, :, t] = c_next
        # todo: 5. 保存y 中的预测
        y[:, :, t] = yt_pred
        # todo: 6. 将缓存追加到 caches 中（约 1 行）
        caches.append(cache)
    ### 代码至此结束 ###

    # 将反向传播所需的值存储在缓存中
    caches = (caches, x)

    return a, y, c, caches

In [9]:
np.random.seed(1)
x_tmp = np.random.randn(3, 10, 7)
a0_tmp = np.random.randn(5, 10)
parameters_tmp = {}
parameters_tmp['Wf'] = np.random.randn(5, 5 + 3)
parameters_tmp['bf'] = np.random.randn(5, 1)
parameters_tmp['Wi'] = np.random.randn(5, 5 + 3)
parameters_tmp['bi']= np.random.randn(5, 1)
parameters_tmp['Wo'] = np.random.randn(5, 5 + 3)
parameters_tmp['bo'] = np.random.randn(5, 1)
parameters_tmp['Wc'] = np.random.randn(5, 5 + 3)
parameters_tmp['bc'] = np.random.randn(5, 1)
parameters_tmp['Wy'] = np.random.randn(2, 5)
parameters_tmp['by'] = np.random.randn(2, 1)

a_tmp, y_tmp, c_tmp, caches_tmp = lstm_forward(x_tmp, a0_tmp, parameters_tmp)
print("a[4][3][6] = ", a_tmp[4][3][6])
print("a.shape = ", a_tmp.shape)
print("y[1][4][3] =", y_tmp[1][4][3])
print("y.shape = ", y_tmp.shape)
print("caches[1][1][1] =\n", caches_tmp[1][1][1])
print("c[1][2][1]", c_tmp[1][2][1])
print("len(caches) = ", len(caches_tmp))

# UNIT TEST    
lstm_forward_test(lstm_forward)

a[4][3][6] =  0.17211776753291663
a.shape =  (5, 10, 7)
y[1][4][3] = 0.9508734618501101
y.shape =  (2, 10, 7)
caches[1][1][1] =
 [ 0.82797464  0.23009474  0.76201118 -0.22232814 -0.20075807  0.18656139
  0.41005165]
c[1][2][1] -0.8555449167181983
len(caches) =  2
All tests passed


**Expected Output**:

```text
a[4][3][6] =  0.172117767533
a.shape =  (5, 10, 7)
y[1][4][3] = 0.95087346185
y.shape =  (2, 10, 7)
caches[1][1][1] =
 [ 0.82797464  0.23009474  0.76201118 -0.22232814 -0.20075807  0.18656139
  0.41005165]
c[1][2][1] -0.855544916718
len(caches) =  2
```

### 恭喜！

现在已经为基本 RNN 和 LSTM 实现了前向传递。使用深度学习框架时，实现前向传递足以构建性能卓越的系统。框架会处理剩下的事情。

应该记住：

* LSTM 与 RNN 类似，它们都使用隐藏状态来传递信息，但 LSTM 还使用单元状态（类似于长期记忆）来帮助处理梯度消失问题。
* LSTM 单元由一个单元状态（或长期记忆）、一个隐藏状态（或短期记忆）以及三个不断更新其输入相关性的门组成：
* 一个<b>遗忘</b>门，它决定哪些输入单元应该被记住并传递。它是一个值在 0 到 1 之间的张量。
* 如果一个单元的值接近于 0，LSTM 会“忘记”前一个单元状态中存储的状态。
* 如果值接近 1，LSTM 通常会记住相应的值。
* 一个<b>更新</b>门，同样是一个包含 0 到 1 之间值的张量。它决定丢弃哪些信息，以及添加哪些新信息。
* 当更新门中的单元接近 1 时，其候选值将传递到隐藏状态。
* 当更新门中的单元接近 0 时，它将被阻止传递到隐藏状态。
* 还有一个<b>输出</b>门，它决定将什么作为时间步的输出发送。
